<a href="https://www.kaggle.com/code/adrienmorel97/v1-exploratory-baseline?scriptVersionId=226479736" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [3]:
# Load the datasets
train_df = pd.read_csv('/kaggle/input/playground-series-s5e3/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e3/test.csv')

In [4]:
# Save 'id' column for submission
test_ids = test_df['id']

# Define the target column
target_column = 'rainfall'

# Select categorical and numerical columns (initial)
numerical_columns = train_df.select_dtypes(exclude=['object']).columns

# Print out column information
print("Target Column:", target_column)
print("\nNumerical Columns:", numerical_columns.tolist())

Target Column: rainfall

Numerical Columns: ['id', 'day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed', 'rainfall']


In [5]:
def date(df):
    df['day_sin'] = np.sin(2 * np.pi * df['day'] / 365)  
    df['day_cos'] = np.cos(2 * np.pi * df['day'] / 365)
    return df

# Apply the date function to both datasets
train_df = date(train_df)
test_df = date(test_df)

In [6]:
# Split train data into features and target
X = train_df.drop(columns=[target_column,'id', 'day'])
y = train_df[target_column]

In [7]:
from sklearn.preprocessing import StandardScaler

preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

X_processed = preprocessor.fit_transform(X)
test_processed = preprocessor.transform(test_df.drop(columns=['id', 'day']))

In [8]:
clf = LogisticRegression()
clf.fit(X_processed, y)

LogisticRegression()

In [9]:
# Make predictions on the test set
test_predictions = clf.predict(test_processed)

# Prepare submission file
submission = pd.DataFrame({'id': test_df['id'], 'rainfall': test_predictions})
submission.to_csv("submission.csv", index=False)